In [1]:
import pandas as pd
import numpy as np
import statsmodels

In [2]:
reviews = [[('../data/reviewer_comparison/H_KA-115225/H_KA-115225_reviewed.bed.formated', False), ('../data/reviewer_comparison/H_KA-115225/H_KA-115225_reviewed.lmt.bed', True), ('../data/reviewer_comparison/H_KA-115225/H_KA-115225_reviewed_Erica.txt', True)],
           [('../data/reviewer_comparison/H_ML-08-0075-202-1121768/H_ML-08-0075-202-1121768_reviewed.bed.formated', False), ('../data/reviewer_comparison/H_ML-08-0075-202-1121768/H_ML-08-0075-202-1121768_reviewed.lmt.bed', True), ('../data/reviewer_comparison/H_ML-08-0075-202-1121768/H_ML-08-0075-202-1121768_Erica.txt', True) ],
           [('../data/reviewer_comparison/H_ND-006G-C117D/H_ND-006G-C117D_reviewed.bed.formated', False), ('../data/reviewer_comparison/H_ND-006G-C117D/H_ND-006G-C117D_reviewed.lmt.bed', True), ('../data/reviewer_comparison/H_ND-006G-C117D/H_ND-006G-C117D_reviewed_Erica.txt', True)],
           [('../data/reviewer_comparison/H_SU-B12-17/H_SU-B12-17_reviewed.bed.formated', False), ('../data/reviewer_comparison/H_SU-B12-17/H_SU-B12-17_reviewed.lmt.bed', True), ('../data/reviewer_comparison/H_SU-B12-17/H_SU-B12-17_reviewed_Erica.txt', True)]]


In [3]:
def get_review_df(filepath, header):
    if header:
        review = pd.read_csv(filepath, sep='\t')
    else:
        col_names = ['Chromosome', 'Start', 'Stop', 'Reference', 'Variant', 
                     'Call', 'Tags', 'Notes']
        review = pd.read_csv(filepath, sep='\t', names=col_names)
    review.drop(['Tags', 'Notes'], inplace=True, axis=1)
    #verify below mapping with preprocessing
    review.replace({'Call': {'O':'F', 'V':'G', 'LOH': 'S', 'LQ': 'F'}}, 
                    inplace=True)
    review.sort_values(['Chromosome', 'Start', 'Stop',
                        'Reference', 'Variant'])
    return review.drop_duplicates(subset=['Chromosome', 'Start', 'Stop', 
                                          'Reference', 'Variant'])

In [4]:
calls = pd.DataFrame()
def get_kappa_counts(list_of_dataframes):
    calls = pd.concat(list_of_dataframes, ignore_index=True)
    kappa_list = []
    for name, group in calls.groupby(['Chromosome', 'Start', 'Stop', 
                                      'Reference', 'Variant']):
        A=F=G=S=0 
        for call in group.Call.iteritems():
            if call[1] == 'A':
                A+=1
            elif call[1] == 'F':
                F+=1
            elif call[1] == 'G':
                G+=1
            elif call[1] == 'S':
                S+=1
        if A+F+G+S > 3:
            raise ValueError('{0}'.format(name))
        kappa_list.append([A, F, G, S])
    return np.array(kappa_list)

In [5]:
kappa_table = np.array([]).reshape(0,4)
for sample in reviews:
    kappa_table = np.concatenate([kappa_table, 
                                  get_kappa_counts([get_review_df(r[0], r[1])
                                                    for r in sample])])

In [6]:
def calculate_kappa(table):
    """https://en.wikipedia.org/wiki/Fleiss%27_kappa"""
    N, k = table.shape

    n = table.sum(axis=1)[0]

    p = table.sum(axis=0)/ (N * n)

    P = (1/(n*(n-1)))*((table**2).sum(axis=1) - n)

    p_bar = P.sum()/N

    P_e = (p**2).sum()

    kappa = (p_bar - P_e)/ (1 - P_e)
    
    return kappa

## Interpreting Fleiss's kappa

| 𝛋        | Interpretation           |
| :-------------: |-------------:|
| < 0      | Poor agreement |
| 0.01 – 0.20      | Slight agreement      |
| 0.21 – 0.40 | Fair agreement      |
| 0.41 – 0.60 | Moderate agreement      |
| 0.61 – 0.80 | Substantial agreement      |
| 0.81 – 1.00 | Almost perfect agreement      |

In [7]:
print('Fleiss\'s kappa statistic for all calls is {0}'.format(calculate_kappa(kappa_table)))

Fleiss's kappa statistic for all calls is 0.3016095617529881


In [28]:
somatic_fail_kappa_table = np.concatenate((kappa_table[:,:3].sum(axis=1).reshape((176,1)), 
                                           kappa_table[:,3:]), axis=1)

In [29]:
print('Fleiss\'s kappa statistic for somatic vs fail is {0}'.format(calculate_kappa(somatic_fail_kappa_table)))

Fleiss's kappa statistic for somatic vs fail is 0.4053260361743308
